## Classificação Binária
***

Essa base de dados foi tirada do site: https://archive.ics.uci.edu/ml/index.php

O Objeto é gerar um diagnostico, ou seja, se o tumor é maligno ou beligno.

Temos 30 Atributos previsores tirados do centro do núcleo da celula: radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, fractal dimension.

Com base nesses atributos vou descobrir se uma determinada imagem tem uma alta probabilidade ou não de ser um tumor maligno.

Arquivos:

* **entradas-breast**: São os atributos previsores

* **saidas-breast**: É a classe de saída (1 - maligno ou  0 - beligno)

***
### Validação Cruzada
***

Para fazer avaliação de um algoritmo, são necessárias 2 bases de dados diferentes (treinamento e teste), a base de treinamento é para gerar o modelo de treinamento e a de teste é para testar esse modelo. Em base de dados grandes 90% é usado para treinar e 10% para teste, já em bases pequenas 75% é usado para treinar e 25% teste.

As vezes tem alguns dados na base de dados de teste que seria melhor estarem na base de dados de treinamento, por isso foi criado o **K-Fold Cross Validation**, é uma técnica mais eficiente para fazer a avaliação do algoritmo e é a tecnica mais utilizada para trabalhos cientificos, ao inves de quebrar a base de dados em 2 partes (treinamento e teste) iremos quebrar em 4 ou mais partes, desses 4 pedaços 3 vai para treinamento e 1 para teste.

Na próxima etapa, iremos pegar outras 3 partes da base de dados e selecionar outra para teste, e assim até todas as 4 partes forem usadas para teste. Em muitos casos eles utiliza K=10 ao inves de 4 partes.

Com isso o resultado final de percentual de acerto é a média de cada uma das partes testada.

***
### Overfitting e Underfitting
***

Quando temos problemas bastante complexos e queremos resolver ele com algo bem simple, chamamos de **underfitting**, ou seja, estamos substimando o problema, já quando temos um problema bem simples e queremos resolver de uma forma complexa, ou seja, utilizando muitos recursos, é o **overfitting**.

#### Underfitting

![img](https://user-images.githubusercontent.com/14116020/56462431-4036f380-6399-11e9-9d25-d91cc07196e8.png)

No primeiro gráfico vimos o modelo ideal para resolver o problema, já no segundo temos um underfitting, ou seja, o problema é relativamente complexo e foi utilizado uma reta para resolve-lo, ou seja, o problema está sendo subestimado.

Normalmente vai ter resultados ruins na base de treinamento

#### Overfitting

![img](https://user-images.githubusercontent.com/14116020/56462459-c5220d00-6399-11e9-886d-b9531fe0e969.png)

Da mesma forma no primeiro gráfico temos o modelo ideal para resolver o problema, já no segundo foi utilizado recursos e algoritmos mais complexos para resolver o mesmo problema, ou seja, está desperdiçando recursos atoa e o algoritmo fica viciado nos dados, se colocar um novo dado é capaz do algoritmo errar.

Vai ter resultados bons na base de treinamento, porém terá resultados ruins na base de teste

É muito especifico, ou seja, memorizado e causa erros na variação de novas instâncias.

Para melhorar o overfitting utilizamos uma técnica chamada **dropout** que serve para zerar alguns valores da camada de entrada aleatoriamente, esses valores zerados não terão influência no resultado final, porém faz com que o resultado não se vicie nos dados de entrada, evitando o overfitting, é recomendado fazer o dropout entre 10% a 20% dos valores de entrada.

Se passar dessa porcentagem sua rede neural pode cair no **underfitting**.

***
### Ajuste do parâmetros (Tuning)
***

Vamos usar uma técnica para que seja descoberta o conjunto de parâmetros que melhor se encaixa em uma rede neural, por exemplo, quantos neuronios será colocado em cada camada escondida?, qual a loss_function que iremos utilizar? como será inicializada os pesos?, qual função de ativação iremos utilizar.

In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.metrics import confusion_matrix, accuracy_score
# Classe para criação da rede neural
from keras.models import Sequential, model_from_json
# Camada de rede neural profunda, cada um dos neuronios é ligada a cada um da camada subsequente.
# Conexão total entre os neuronios.
from keras.layers import Dense, Dropout
# Para validação cruzada
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
# Tunning de pesquisa em grade para descobrir os melhores parâmetros
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
previsores = pd.read_csv('./arquivos/entradas-breast.csv')

In [3]:
classes = pd.read_csv('./arquivos/saidas-breast.csv')

In [4]:
def criar_rede_neural(optimizer='adam',
                      loss_function='binary_crossentropy',
                      kernel_initializer='normal',
                      activation='relu',
                      neurons=16,
                      dropout=0.2):
    """
    Função responsável por criar a rede neural
    """
    
    # Criando uma nova rede neural
    classificador = Sequential()
    
    # Adicionar a primeira camada oculta.
    # UNITS = Quantidade de neuronios que farão parte da minha camada oculta
    # (número de entradas (colunas) + numero de classes de saída (binarios = 1))/2
    # ACTIVATION = Função de ativação
    # KERNEL_INITIALIZER = Como você vai fazer a inicialização dos pesos (randomica)
    # INPUT_DIM = Quantos elementos existem na primeira camada de entrada
    entrada = 30
    classificador.add(Dense(
        units = neurons,
        activation = activation,
        kernel_initializer = kernel_initializer,
        input_dim = entrada
    ))
    
    # Dropout da primeira camada, zera 20% da camada de entrada
    classificador.add(Dropout(dropout))
    
    # Segunda camada oculta
    classificador.add(Dense(
        units = neurons,
        activation = activation,
        kernel_initializer = kernel_initializer
    ))
    
    # Dropout da segunda camada oculta, zera 20% da entrada
    classificador.add(Dropout(dropout))
    
    # Adicionar a camada de saída
    neurons_binary = 1
    classificador.add(Dense(
        units = neurons_binary,
        activation = 'sigmoid'
    ))
    
    # Modificar os parâmetros do otimizados para melhorar o treinamento (não ta sendo usado)
    # LR = Learning Rate, Taxa de aprendizagem, padrão 0.001, quanto maior o lr
    # mais rapido o erro chega no mínimo global, porém se tiver muito alto ele pode acaber passando.
    # DECAY = Função de decaimento do valor da taxa de aprendizagem, padrão 0.0, ou seja, não diminui
    # CLIPVALUE = Ele prende o valor de erro entre 0.5 e -0.5
    otimizador = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
    
    # Vamos compilar a nossa rede neural criada
    # OPTIMIZER = Função para fazer o ajuste dos pesos para otimização da decida do gradiente, 'adam'
    # LOSS = Função de perda, ou seja, maneira de fazer o cálculo do erro.
    # METRICS = Métricas para avaliação dos resultados (binary_accuracy - pega qtd de registros certos e errados)
    classificador.compile(
        optimizer = optimizer,
        loss = loss_function,
        metrics = ['binary_accuracy']
    )
    
    return classificador

In [5]:
def pegar_melhores_parametros():
    """
    Pega os melhores parâmetros para o treinamento.
    """
    
    # Criar o classificador com validação cruzada
    # BUILD_FB = Função de criação da rede neural
    classificador = KerasClassifier(build_fn = criar_rede_neural)
    
    # Possíveis parâmetros para treinamento
    # EPOCHS = Número de vezes que irá rodar todo o processo de atualização de pesos e melhoria
    # da porcentagem de acerto.
    # BATCH_SIZE = Vai calcular o erro e atualizar os pesos de N em N registros
    parametros = {
        'batch_size': [10, 30],
        'epochs': [50, 100],
        'optimizer': ['adam', 'sgd'],
        'loss_function': ['binary_crossentropy', 'hinge'],
        'kernel_initializer': ['random_uniform', 'normal'],
        'activation': ['relu', 'tanh'],
        'neurons': [16, 8],
        'dropout': [0.2, 0.1]
    }
    
    # Fazer a busca dos melhores parâmetros para avaliarmos qual é o melhor parâmetro para treinamento
    # ESTIMATOR = Nosso classificador de treinamento
    # PARAM_GRID = Possíveis parâmetros para o treinamento
    # SCORING = Como eu quero fazer a avaliação dos resultados
    # CV = Quantidade de pedaços que será quebrado a base de dados da validação cruzada
    busca = GridSearchCV(
        estimator = classificador,
        param_grid = parametros,
        scoring = 'accuracy',
        cv = 5
    )
    
    # Executar a busca pelos melhores parâmetros
    busca_concluida = busca.fit(previsores, classes)
    
    melhores_parametros = busca_concluida.best_params_
    # {10, 100, 'adam', 'binary_crossentropy', 'normal', 'relu', 8, 0.2}
    melhor_precisao = busca_concluida.best_score_
    # 0.9253245...
    
    return melhores_parametros, melhor_precisao

In [6]:
def realiza_teste_da_rede_neural():
    """
    Faz a suite de testes com a rede neural utilizando 
    """
    
    # Criar o classificador com validação cruzada
    # BUILD_FB = Função de criação da rede neural
    # EPOCHS = Número de vezes que irá rodar todo o processo de atualização de pesos e melhoria
    # da porcentagem de acerto.
    # BATCH_SIZE = Vai calcular o erro e atualizar os pesos de 10 em 10 registros
    classificador = KerasClassifier(
        build_fn = criar_rede_neural,
        epochs = 100,
        batch_size = 10
    )
    
    # Realiza e pegar o resultado do treinamento dos 10 ciclos
    # ESTIMATOR = Nosso classificador de treinamento
    # X = Atributos previsores, ou seja, as entradas
    # y = Classes de saída
    # CV = Quantidade de pedaços que será quebrado a base de dados da validação cruzada
    # SCORING = Como será retornado os resultados
    resultados = cross_val_score(
        estimator = classificador,
        X = previsores,
        y = classes,
        cv = 10,
        scoring = 'accuracy'
    )
    
    # Acerto final
    media = resultados.mean()

    # Desvio padrão, ou seja, quanto os valores estão variando em relação a média
    # Qual longe ou perto os 10 valores gerados estão da média
    # Quanto maior o valor, mais a tendencia de ter overfitting,
    # ou seja, seu algoritmo se adaptou demais aos seus dados
    # Quando passar novos dados ela não terá bons resultados.
    desvio = resultados.std()

    return resultados, media, desvio

In [7]:
# Verificar o modelo em um algoritmo de teste
resultados, media, desvio = realiza_teste_da_rede_neural()

Epoch 1/100
512/512 [==============================] - 1s 1ms/step - loss: 1.2070 - binary_accuracy: 0.6113
Epoch 2/100
512/512 [==============================] - 0s 173us/step - loss: 0.5715 - binary_accuracy: 0.6699
Epoch 3/100
512/512 [==============================] - 0s 160us/step - loss: 0.6574 - binary_accuracy: 0.6445
Epoch 4/100
512/512 [==============================] - 0s 198us/step - loss: 0.5411 - binary_accuracy: 0.6895
Epoch 5/100
512/512 [==============================] - 0s 192us/step - loss: 0.5768 - binary_accuracy: 0.6875
Epoch 6/100
512/512 [==============================] - 0s 182us/step - loss: 0.4862 - binary_accuracy: 0.7520
Epoch 7/100
512/512 [==============================] - 0s 190us/step - loss: 0.4825 - binary_accuracy: 0.7695
Epoch 8/100
512/512 [==============================] - 0s 189us/step - loss: 0.4654 - binary_accuracy: 0.7754
Epoch 9/100
512/512 [==============================] - 0s 193us/step - loss: 0.5147 - binary_accuracy: 0.8008
Epoch 10/100

512/512 [==============================] - 0s 173us/step - loss: 0.1801 - binary_accuracy: 0.9316
Epoch 75/100
512/512 [==============================] - 0s 204us/step - loss: 0.1726 - binary_accuracy: 0.9355
Epoch 76/100
512/512 [==============================] - 0s 206us/step - loss: 0.1625 - binary_accuracy: 0.9453
Epoch 77/100
512/512 [==============================] - 0s 180us/step - loss: 0.1794 - binary_accuracy: 0.9375
Epoch 78/100
512/512 [==============================] - 0s 191us/step - loss: 0.2212 - binary_accuracy: 0.9160
Epoch 79/100
512/512 [==============================] - 0s 215us/step - loss: 0.1580 - binary_accuracy: 0.9434
Epoch 80/100
512/512 [==============================] - 0s 187us/step - loss: 0.1768 - binary_accuracy: 0.9297
Epoch 81/100
512/512 [==============================] - 0s 173us/step - loss: 0.1869 - binary_accuracy: 0.9258
Epoch 82/100
512/512 [==============================] - 0s 213us/step - loss: 0.1581 - binary_accuracy: 0.9355
Epoch 83/100
5

Epoch 48/100
512/512 [==============================] - 0s 180us/step - loss: 0.2554 - binary_accuracy: 0.8945
Epoch 49/100
512/512 [==============================] - 0s 194us/step - loss: 0.2637 - binary_accuracy: 0.9062
Epoch 50/100
512/512 [==============================] - 0s 179us/step - loss: 0.2202 - binary_accuracy: 0.9043 0s - loss: 0.2109 - binary_accuracy: 0.915
Epoch 51/100
512/512 [==============================] - 0s 168us/step - loss: 0.2757 - binary_accuracy: 0.8809
Epoch 52/100
512/512 [==============================] - 0s 171us/step - loss: 0.2767 - binary_accuracy: 0.8750
Epoch 53/100
512/512 [==============================] - 0s 167us/step - loss: 0.2607 - binary_accuracy: 0.8945
Epoch 54/100
512/512 [==============================] - 0s 180us/step - loss: 0.2361 - binary_accuracy: 0.9102
Epoch 55/100
512/512 [==============================] - 0s 189us/step - loss: 0.2341 - binary_accuracy: 0.8926
Epoch 56/100
512/512 [==============================] - 0s 168us/step

512/512 [==============================] - 0s 212us/step - loss: 0.5073 - binary_accuracy: 0.7031
Epoch 22/100
512/512 [==============================] - 0s 192us/step - loss: 0.4775 - binary_accuracy: 0.7363
Epoch 23/100
512/512 [==============================] - 0s 178us/step - loss: 0.4835 - binary_accuracy: 0.7461
Epoch 24/100
512/512 [==============================] - 0s 215us/step - loss: 0.4904 - binary_accuracy: 0.7285
Epoch 25/100
512/512 [==============================] - 0s 197us/step - loss: 0.4800 - binary_accuracy: 0.7461
Epoch 26/100
512/512 [==============================] - 0s 189us/step - loss: 0.4960 - binary_accuracy: 0.7637
Epoch 27/100
512/512 [==============================] - 0s 190us/step - loss: 0.4733 - binary_accuracy: 0.7363
Epoch 28/100
512/512 [==============================] - 0s 176us/step - loss: 0.4722 - binary_accuracy: 0.7617
Epoch 29/100
512/512 [==============================] - 0s 180us/step - loss: 0.4642 - binary_accuracy: 0.7637
Epoch 30/100
5

512/512 [==============================] - 0s 191us/step - loss: 0.2326 - binary_accuracy: 0.8926
Epoch 95/100
512/512 [==============================] - 0s 199us/step - loss: 0.2177 - binary_accuracy: 0.9023
Epoch 96/100
512/512 [==============================] - 0s 180us/step - loss: 0.2051 - binary_accuracy: 0.9023
Epoch 97/100
512/512 [==============================] - 0s 161us/step - loss: 0.2092 - binary_accuracy: 0.9141
Epoch 98/100
512/512 [==============================] - 0s 176us/step - loss: 0.2096 - binary_accuracy: 0.9102
Epoch 99/100
512/512 [==============================] - 0s 166us/step - loss: 0.2054 - binary_accuracy: 0.9160
Epoch 100/100
512/512 [==============================] - 0s 176us/step - loss: 0.2326 - binary_accuracy: 0.9004
Epoch 1/100
512/512 [==============================] - 0s 617us/step - loss: 1.0704 - binary_accuracy: 0.5703
Epoch 2/100
512/512 [==============================] - 0s 176us/step - loss: 0.7190 - binary_accuracy: 0.5918
Epoch 3/100
512

...

Epoch 62/100
512/512 [==============================] - 0s 194us/step - loss: 0.2572 - binary_accuracy: 0.8828
Epoch 63/100
512/512 [==============================] - 0s 205us/step - loss: 0.2326 - binary_accuracy: 0.9102
Epoch 64/100
512/512 [==============================] - 0s 197us/step - loss: 0.2518 - binary_accuracy: 0.8926
Epoch 65/100
512/512 [==============================] - 0s 211us/step - loss: 0.2266 - binary_accuracy: 0.9023
Epoch 66/100
512/512 [==============================] - 0s 182us/step - loss: 0.2089 - binary_accuracy: 0.9082
Epoch 67/100
512/512 [==============================] - 0s 189us/step - loss: 0.2154 - binary_accuracy: 0.9023
Epoch 68/100
512/512 [==============================] - 0s 187us/step - loss: 0.2138 - binary_accuracy: 0.9180
Epoch 69/100
512/512 [==============================] - 0s 225us/step - loss: 0.2408 - binary_accuracy: 0.8887
Epoch 70/100
512/512 [==============================] - 0s 212us/step - loss: 0.2132 - binary_accuracy: 0.9102
E

512/512 [==============================] - 0s 174us/step - loss: 0.2694 - binary_accuracy: 0.8926
Epoch 36/100
512/512 [==============================] - 0s 166us/step - loss: 0.3341 - binary_accuracy: 0.8496
Epoch 37/100
512/512 [==============================] - 0s 185us/step - loss: 0.3164 - binary_accuracy: 0.8633
Epoch 38/100
512/512 [==============================] - 0s 171us/step - loss: 0.3077 - binary_accuracy: 0.8613
Epoch 39/100
512/512 [==============================] - 0s 160us/step - loss: 0.3176 - binary_accuracy: 0.8555
Epoch 40/100
512/512 [==============================] - 0s 170us/step - loss: 0.3007 - binary_accuracy: 0.8789
Epoch 41/100
512/512 [==============================] - 0s 177us/step - loss: 0.3413 - binary_accuracy: 0.8477
Epoch 42/100
512/512 [==============================] - 0s 160us/step - loss: 0.3013 - binary_accuracy: 0.8672
Epoch 43/100
512/512 [==============================] - 0s 182us/step - loss: 0.2781 - binary_accuracy: 0.8867
Epoch 44/100
5

512/512 [==============================] - 0s 189us/step - loss: 0.5019 - binary_accuracy: 0.7344
Epoch 9/100
512/512 [==============================] - 0s 188us/step - loss: 0.5125 - binary_accuracy: 0.7266
Epoch 10/100
512/512 [==============================] - 0s 249us/step - loss: 0.4520 - binary_accuracy: 0.7793
Epoch 11/100
512/512 [==============================] - 0s 184us/step - loss: 0.4376 - binary_accuracy: 0.7793
Epoch 12/100
512/512 [==============================] - 0s 173us/step - loss: 0.5086 - binary_accuracy: 0.7578
Epoch 13/100
512/512 [==============================] - 0s 179us/step - loss: 0.4324 - binary_accuracy: 0.7754
Epoch 14/100
512/512 [==============================] - 0s 186us/step - loss: 0.4408 - binary_accuracy: 0.7969
Epoch 15/100
512/512 [==============================] - 0s 216us/step - loss: 0.4293 - binary_accuracy: 0.7969
Epoch 16/100
512/512 [==============================] - 0s 178us/step - loss: 0.4136 - binary_accuracy: 0.7891
Epoch 17/100
51

512/512 [==============================] - 0s 187us/step - loss: 0.2995 - binary_accuracy: 0.8594
Epoch 82/100
512/512 [==============================] - 0s 178us/step - loss: 0.2681 - binary_accuracy: 0.8789
Epoch 83/100
512/512 [==============================] - 0s 190us/step - loss: 0.2351 - binary_accuracy: 0.8945
Epoch 84/100
512/512 [==============================] - 0s 189us/step - loss: 0.2684 - binary_accuracy: 0.8770
Epoch 85/100
512/512 [==============================] - 0s 187us/step - loss: 0.2841 - binary_accuracy: 0.8809
Epoch 86/100
512/512 [==============================] - 0s 197us/step - loss: 0.2656 - binary_accuracy: 0.9004
Epoch 87/100
512/512 [==============================] - 0s 177us/step - loss: 0.2513 - binary_accuracy: 0.8984
Epoch 88/100
512/512 [==============================] - 0s 195us/step - loss: 0.2858 - binary_accuracy: 0.8867
Epoch 89/100
512/512 [==============================] - 0s 197us/step - loss: 0.2361 - binary_accuracy: 0.8965
Epoch 90/100
5

Epoch 55/100
513/513 [==============================] - 0s 203us/step - loss: 0.2945 - binary_accuracy: 0.8811
Epoch 56/100
513/513 [==============================] - 0s 182us/step - loss: 0.2899 - binary_accuracy: 0.8713
Epoch 57/100
513/513 [==============================] - 0s 181us/step - loss: 0.2524 - binary_accuracy: 0.8947
Epoch 58/100
513/513 [==============================] - 0s 184us/step - loss: 0.2827 - binary_accuracy: 0.8791
Epoch 59/100
513/513 [==============================] - 0s 218us/step - loss: 0.3131 - binary_accuracy: 0.8674
Epoch 60/100
513/513 [==============================] - 0s 240us/step - loss: 0.3260 - binary_accuracy: 0.8655
Epoch 61/100
513/513 [==============================] - 0s 239us/step - loss: 0.3133 - binary_accuracy: 0.8655
Epoch 62/100
513/513 [==============================] - 0s 258us/step - loss: 0.2969 - binary_accuracy: 0.8850
Epoch 63/100
513/513 [==============================] - 0s 226us/step - loss: 0.3034 - binary_accuracy: 0.8908
E

In [8]:
print(resultados)

[0.75438596 0.9122807  0.9122807  0.8245614  0.96491228 0.94736842
 0.94736842 0.94736842 0.85964912 0.92857143]


In [9]:
print(media)

0.899874686716792


In [10]:
print(desvio)

0.0637066160126887


In [11]:
# Treinar o algoritmo para produção
# EPOCHS = Número de vezes que irá rodar todo o processo de atualização de pesos e melhoria
# da porcentagem de acerto.
# BATCH_SIZE = Vai calcular o erro e atualizar os pesos de 10 em 10 registros
classificador = criar_rede_neural(neurons=8)
classificador.fit(
    previsores,
    classes,
    batch_size = 10,
    epochs = 100
)

Epoch 1/100
569/569 [==============================] - 0s 583us/step - loss: 5.3311 - binary_accuracy: 0.5712
Epoch 2/100
569/569 [==============================] - 0s 167us/step - loss: 1.3943 - binary_accuracy: 0.5448
Epoch 3/100
569/569 [==============================] - 0s 165us/step - loss: 0.8955 - binary_accuracy: 0.6520
Epoch 4/100
569/569 [==============================] - 0s 166us/step - loss: 0.7264 - binary_accuracy: 0.6432
Epoch 5/100
569/569 [==============================] - 0s 179us/step - loss: 0.6148 - binary_accuracy: 0.6169
Epoch 6/100
569/569 [==============================] - 0s 186us/step - loss: 0.6168 - binary_accuracy: 0.6731
Epoch 7/100
569/569 [==============================] - 0s 159us/step - loss: 0.6383 - binary_accuracy: 0.7329
Epoch 8/100
569/569 [==============================] - 0s 189us/step - loss: 0.5300 - binary_accuracy: 0.7504
Epoch 9/100
569/569 [==============================] - 0s 190us/step - loss: 0.5109 - binary_accuracy: 0.7557
Epoch 10/1

569/569 [==============================] - 0s 172us/step - loss: 0.3163 - binary_accuracy: 0.8682
Epoch 75/100
569/569 [==============================] - 0s 175us/step - loss: 0.3444 - binary_accuracy: 0.8453
Epoch 76/100
569/569 [==============================] - 0s 200us/step - loss: 0.3211 - binary_accuracy: 0.8436
Epoch 77/100
569/569 [==============================] - 0s 196us/step - loss: 0.2827 - binary_accuracy: 0.8752
Epoch 78/100
569/569 [==============================] - 0s 171us/step - loss: 0.2789 - binary_accuracy: 0.8717
Epoch 79/100
569/569 [==============================] - 0s 191us/step - loss: 0.2957 - binary_accuracy: 0.8436
Epoch 80/100
569/569 [==============================] - 0s 185us/step - loss: 0.3029 - binary_accuracy: 0.8612
Epoch 81/100
569/569 [==============================] - 0s 173us/step - loss: 0.2584 - binary_accuracy: 0.8787
Epoch 82/100
569/569 [==============================] - 0s 172us/step - loss: 0.3030 - binary_accuracy: 0.8576
Epoch 83/100
5

In [12]:
# Salvando a estrutura da rede neural
classificador_json = classificador.to_json()
with open('classificador.json', 'w') as json_file:
    json_file.write(classificador_json)

In [13]:
# Salvando os pesos da rede neural (pip install h5py)
classificador.save_weights('classificador.h5')

***

In [14]:
# Pegando a estrutura da rede neural
with open('classificador.json', 'r') as json_file:
    estrutura = json_file.read()

In [15]:
# Criando o classificador
classificador_externo = model_from_json(estrutura)

In [16]:
# Pegando os pesos da rede neural
classificador_externo.load_weights('classificador.h5')

In [17]:
# Criando um novo registro
novo_registro = np.array([
    [
        15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
        0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
        0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
        0.84, 158, 0.363
    ]
])

In [18]:
# Verificar se há tumor maligno nos novos registros
previsao = classificador_externo.predict(novo_registro)
print(previsao)

[[1.]]


In [19]:
# Verificar se existe tumor maligno nos novos registros
previsao = (previsao > 0.5)
if True in previsao:
    print("Existe tumor maligno")
else:
    print("Não eixste tumor maligno")

Existe tumor maligno


***

In [20]:
# Vamos fazer alguns testes com nossa base de dados de teste
# Usaremos a mesma base de dados de treinamento o que não é o ideal
from sklearn.metrics import confusion_matrix, accuracy_score

In [21]:
# Realiza as previsões com a base toda (True = Tem tumor maligno, False = Não tem)
previsoes = classificador_externo.predict(previsores)
previsoes = (previsoes > 0.5)

In [22]:
# A precisão vai dar o mesmo resultado dos testes já realizados
precisao = accuracy_score(classes, previsoes)
print(precisao)

0.9279437609841827


#### Matriz de confusão

Essa matriz pega todos os registros e verifica a quantidade de positivos e falsos positivos.

```
                | tumor beligno | tumor maligno |
| tumor beligno | 197           | 15            |
| tumor maligno | 26            | 331           |
```

In [23]:
# 197 registros classificados como tumores não malignos corretamente
# 15 registros foram classificados como tumores malignos e não eram
# 26 registros foram classificados como tumores não malignos e eram
# 331 registros classificados como tumores malignos corretamente
matriz = confusion_matrix(classes, previsoes)
print(matriz)

[[197  15]
 [ 26 331]]
